# ZOMATO DATA ANALYSIS


In [33]:
import findspark
findspark.init()

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark=SparkSession.builder.appName("Zomato Analysis").getOrCreate()

#### Schema creation for Zomato data

In [48]:

zomato_schema=StructType(fields=[StructField("Restaurant ID", IntegerType(),False),
                                StructField("Restaurant Name", StringType(),True),
                                StructField("Country Code", StringType(),True),
                                StructField("City", StringType(),True),
                                StructField("Address", StringType(),True),
                                StructField("Locality", StringType(),True),
                                StructField("Locality Verbose", StringType(),True),
                                StructField("Longitude", IntegerType(),True),
                                StructField("Latitude", IntegerType(),True),
                                StructField("Cuisines", StringType(),True),
                                StructField("Average Cost for two", IntegerType(),True),
                                StructField("Currency", StringType(),True),
                                StructField("Has Table booking", StringType(),True),
                                StructField("Has Online delivery", StringType(),True),
                                StructField("Is delivering now", StringType(),True),
                                StructField("Switch to order menu", StringType(),True),
                                StructField("Price range", IntegerType(),True),
                                StructField("Aggregate rating", FloatType(),True),
                                StructField("Rating color", StringType(),True),
                                StructField("Rating text", StringType(),True),
                                StructField("Votes", IntegerType(),True)])


#### Reading the zomato file

In [49]:
zomato_df=spark.read\
.option("header",True)\
.schema(zomato_schema)\
.csv("F:\zomato.csv")

In [50]:
zomato_df.show(10,False)

+-------------+----------------------------------------+------------+----------------+-----------------------------------------------------------------------------------+-----------------------------------------------+------------------------------------------------------------+---------+--------+----------------------------------+--------------------+----------------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+
|Restaurant ID|Restaurant Name                         |Country Code|City            |Address                                                                            |Locality                                       |Locality Verbose                                            |Longitude|Latitude|Cuisines                          |Average Cost for two|Currency        |Has Table booking|Has Online delivery|Is delivering now|Switch to order menu|Price range|Aggregate rating|Rating color|R

#### Renaming column names in Zomato DF

In [51]:
zomato_df_new=zomato_df.withColumnRenamed("Restaurant ID","Restaurant_ID")\
.withColumnRenamed("Restaurant Name","Restaurant_Name")\
.withColumnRenamed("Country Code","Country_Code")\
.withColumnRenamed("Locality Verbose","Locality_Verbose")\
.withColumnRenamed("Average Cost for two","Average_Cost_for_two")\
.withColumnRenamed("Has Table booking","Has_Table_booking")\
.withColumnRenamed("Has Online delivery","Has_Online_delivery")\
.withColumnRenamed("Is delivering now","Is_delivering_now")\
.withColumnRenamed("Switch to order menu","order_menu")\
.withColumnRenamed("Price range","Price_range")\
.withColumnRenamed("Aggregate rating","Aggregate_rating")\
.withColumnRenamed("Rating color","Rating_color")\
.withColumnRenamed("Rating text","Rating_text")


In [40]:
zomato_df_new.show(10) 

+-------------+--------------------+------------+----------------+--------------------+--------------------+--------------------+---------+--------+--------------------+--------------------+----------------+-----------------+-------------------+-----------------+----------+-----------+----------------+------------+-----------+-----+
|Restaurant_ID|     Restaurant_Name|Country_Code|            City|             Address|            Locality|    Locality_Verbose|Longitude|Latitude|            Cuisines|Average_Cost_for_two|        Currency|Has_Table_booking|Has_Online_delivery|Is_delivering_now|order_menu|Price_range|Aggregate_rating|Rating_color|Rating_text|Votes|
+-------------+--------------------+------------+----------------+--------------------+--------------------+--------------------+---------+--------+--------------------+--------------------+----------------+-----------------+-------------------+-----------------+----------+-----------+----------------+------------+-----------+--

####  Reading Country Code file

In [35]:

country_code_df=spark.read\
.option("header",True)\
.option("Inferschema",True)\
.csv("F:\countrycode.csv")





In [36]:
country_code_df.show()

+------------+--------------+
|Country Code|       Country|
+------------+--------------+
|           1|         India|
|          14|     Australia|
|          30|        Brazil|
|          37|        Canada|
|          94|     Indonesia|
|         148|   New Zealand|
|         162|   Phillipines|
|         166|         Qatar|
|         184|     Singapore|
|         189|  South Africa|
|         191|     Sri Lanka|
|         208|        Turkey|
|         214|           UAE|
|         215|United Kingdom|
|         216| United States|
+------------+--------------+



#### Renaming Column in  Countrycode DF and Create Sql table 

In [89]:
country_code_df\
.withColumnRenamed("Country Code","Country_Code")\
.createOrReplaceTempView("country_code")


In [54]:
zomato_df_new.createOrReplaceTempView("zomato")

#### Top Countries with best Average Food Rating

In [95]:
spark.sql("""
select c.country,count(z.Restaurant_ID) total_restaurant,round(avg(z.Aggregate_rating),2) avg_rating
from country_code c join zomato z on c.country_code=z.country_code
group by c.Country
order by avg_rating desc

 """).show() 

+--------------+----------------+----------+
|       country|total_restaurant|avg_rating|
+--------------+----------------+----------+
|   Phillipines|              22|      4.47|
|        Turkey|              34|       4.3|
|     Indonesia|              21|       4.3|
|   New Zealand|              40|      4.26|
|           UAE|              60|      4.24|
|  South Africa|              60|      4.21|
|United Kingdom|              80|      4.09|
|         Qatar|              20|      4.06|
| United States|             434|       4.0|
|     Sri Lanka|              20|      3.87|
|        Brazil|              60|      3.76|
|     Australia|              24|      3.66|
|     Singapore|              20|      3.59|
|        Canada|               4|      3.58|
|         India|            8652|      2.52|
+--------------+----------------+----------+



####  Top Restaurants with Good Agg rating outside the India

In [102]:
spark.sql("""
select z.Restaurant_ID,z.Restaurant_Name,c.country,z.City,z.Aggregate_rating
from zomato z join country_code c on z.country_code=c.country_code
where c.Country !="India"
order by Aggregate_rating desc, Restaurant_Name

 """).show() 

+-------------+--------------------+--------------+--------------+----------------+
|Restaurant_ID|     Restaurant_Name|       country|          City|Aggregate_rating|
+-------------+--------------------+--------------+--------------+----------------+
|     18269368|AB's Absolute Bar...|           UAE|         Dubai|             4.9|
|     17375072|Atlanta Highway S...| United States|   Gainesville|             4.9|
|      6127163|                 Bao|United Kingdom|        London|             4.9|
|      7300955|  Braseiro da G��vea|        Brazil|Rio de Janeiro|             4.9|
|     18254160| Carnival By Tresind|           UAE|         Dubai|             4.9|
|      6600427|          Coco Bambu|        Brazil|     Bras�_lia|             4.9|
|      6501534|Cube - Tasting Ki...|  South Africa|    Inner City|             4.9|
|      5927248|    Draft Gastro Pub|        Turkey|     ��stanbul|             4.9|
|      6114829|       Duck & Waffle|United Kingdom|        London|          

#### Top Restaurants in India with Good Agg Ratings

In [112]:
spark.sql("""
select z.Restaurant_ID,z.Restaurant_Name,z.City,z.Aggregate_rating
from zomato z join country_code c on z.country_code=c.country_code
where c.Country ="India"
order by Aggregate_rating desc,z.Restaurant_Name
""").show()

+-------------+--------------------+----------+----------------+
|Restaurant_ID|     Restaurant_Name|      City|Aggregate_rating|
+-------------+--------------------+----------+----------------+
|     18452864|AB's - Absolute B...| Hyderabad|             4.9|
|     18384227|AB's - Absolute B...|   Chennai|             4.9|
|        94286|AB's - Absolute B...| Hyderabad|             4.9|
|      2100702|     Barbeque Nation|  Guwahati|             4.9|
|        20842|     Barbeque Nation|   Kolkata|             4.9|
|      3300958|     Barbeque Nation|    Nagpur|             4.9|
|        25570|     Barbeque Nation|   Kolkata|             4.9|
|      2800856|     Barbeque Nation|     Vizag|             4.9|
|      3001321|             CakeBee|Coimbatore|             4.9|
|     18384115|         Caterspoint|   Gurgaon|             4.9|
|       800468|Grandson of Tunda...|   Lucknow|             4.9|
|         2004|Indian Accent - T...| New Delhi|             4.9|
|     18345728|      Masa

#### Best Restaurants with Good Avg cost for two and with best ratings 

In [111]:
spark.sql("""
select z.Restaurant_ID,z.Restaurant_Name,z.City,z.Average_Cost_for_two,z.Aggregate_rating
from zomato z join country_code c on z.country_code=c.country_code
where c.Country ="India" and Average_Cost_for_two is NOt null and Average_Cost_for_two>0
order by Aggregate_rating desc,Average_Cost_for_two, z.City
""").show()

+-------------+--------------------+----------+--------------------+----------------+
|Restaurant_ID|     Restaurant_Name|      City|Average_Cost_for_two|Aggregate_rating|
+-------------+--------------------+----------+--------------------+----------------+
|       310143|  Naturals Ice Cream| New Delhi|                 150|             4.9|
|      2600109|Sagar Gaire Fast ...|    Bhopal|                 250|             4.9|
|       800468|Grandson of Tunda...|   Lucknow|                 300|             4.9|
|      3001321|             CakeBee|Coimbatore|                 350|             4.9|
|     18384115|         Caterspoint|   Gurgaon|                 500|             4.9|
|     18416632|The Great Indian Pub|  Dehradun|                1500|             4.9|
|      2100702|     Barbeque Nation|  Guwahati|                1500|             4.9|
|        94286|AB's - Absolute B...| Hyderabad|                1500|             4.9|
|     18452864|AB's - Absolute B...| Hyderabad|       

#### Top Cities with best Avg Ratings in India

In [126]:
spark.sql("""
with t1 as(select city,count(restaurant_id) total_restaurants,avg(Aggregate_rating) avg_rating
from zomato
where Country_Code=1 
group by city
order by avg_rating desc)
select * from t1
where total_restaurants>10""").show()


+------------+-----------------+------------------+
|        city|total_restaurants|        avg_rating|
+------------+-----------------+------------------+
|   Bangalore|               20| 4.374999976158142|
|   Hyderabad|               18| 4.344444420602587|
|     Chennai|               20| 4.314999985694885|
|     Kolkata|               20| 4.254999995231628|
|         Goa|               20| 4.245000004768372|
|        Pune|               20| 4.219999980926514|
|     Lucknow|               21|4.1952380793435236|
|    Guwahati|               21| 4.190476167769659|
|   Ahmedabad|               21| 4.161904777799334|
|  Coimbatore|               20| 4.135000002384186|
|      Jaipur|               20| 4.130000030994415|
|      Mumbai|               20| 4.084999978542328|
|       Kochi|               20| 4.079999995231629|
|    Dehradun|               20|              4.05|
|  Chandigarh|               18| 4.049999992052714|
|    Vadodara|               20|             4.025|
|       Viza

#### Top Cities with best Avg Ratings ouside the India

In [127]:
spark.sql("""
with t1 as(select city,count(restaurant_id) total_restaurants,avg(Aggregate_rating) avg_rating
from zomato
where Country_Code!=1 
group by city
order by avg_rating desc)
select * from t1
where total_restaurants>10""").show()

+--------------------+-----------------+------------------+
|                city|total_restaurants|        avg_rating|
+--------------------+-----------------+------------------+
|              London|               20| 4.534999978542328|
|             Orlando|               20| 4.475000023841858|
|           Tampa Bay|               20| 4.409999990463257|
|      Rest of Hawaii|               20| 4.409999966621399|
|               Dubai|               20|4.3700000762939455|
|             Jakarta|               16| 4.356250017881393|
|              Ankara|               20| 4.304999983310699|
|             Sandton|               11| 4.300000039013949|
|           Abu Dhabi|               20|4.2999999761581424|
|           ��stanbul|               14| 4.292857101985386|
|            Auckland|               20| 4.274999988079071|
|      Rio de Janeiro|               20| 4.264999985694885|
|               Boise|               20|4.2600000381469725|
|     Wellington City|               20|